# Table of Contents

* [Introduction](#1)
* [Workspace Preparation](#2)
* [Data Preparation and Features Processing](#3)
* [Getting x_train, x_test, y_train, y_test](#4)
* [MLFlow workspace preparation and Use](#5)
* [Conclusions](#6)
* [References](#7)

# Introduction

> Nota: Este Jupyter Notebook en la mayor medida esta en Ingles. Se encuentra en español las partes del contexto y habra comentarios con spanglish. ;)

### Escenario del Notebook

- Una empresa activa en Big Data y Data Science desea contratar científicos de datos entre las personas que aprueben exitosamente algunos cursos impartidos por la empresa. Muchas personas se inscriben en su capacitación. La empresa quiere saber cuáles de estos candidatos realmente desean trabajar para la empresa después de la capacitación o si están buscando un nuevo empleo, ya que esto ayuda a reducir los costos y el tiempo, así como la calidad de la capacitación o la planificación de los cursos y la categorización de los candidatos. La información relacionada con la demografía, la educación y la experiencia está disponible a partir de la inscripción y matrícula de los candidatos.

- Este conjunto de datos está diseñado para comprender los factores que llevan a una persona a dejar su trabajo actual, también para investigaciones de recursos humanos. Utilizando el modelo o modelos que utilicen las credenciales actuales, la demografía y los datos de experiencia, **se predecirá la probabilidad de que un candidato busque un nuevo empleo o trabaje para la empresa, además de interpretar los factores que afectan la decisión del empleado.** 

- Todos los datos se dividen en conjuntos de entrenamiento y prueba. 

- El objetivo no está incluido en la prueba, pero se cuenta con el archivo de datos de los valores objetivo de prueba para tareas relacionadas. 

- También se proporciona una muestra de envío correspondiente al identificador de inscrito en el conjunto de prueba con las columnas: enrollee_id, target.


# Workspace preparation

In [1]:
#%matplotlib inline
#%config Completer.use_jedi=False

In [2]:
from pathlib import Path
from urllib.parse import urlparse

import pandas as pd
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import (accuracy_score, precision_score, recall_score, f1_score,
                             roc_auc_score, classification_report)
import mlflow
import mlflow.sklearn

# Data Preparation and Features Processing

In [3]:
data = pd.read_csv(Path("../data/aug_train.csv"))
targets = data[["target"]]
data.drop(["enrollee_id", "target"], inplace=True, axis=1)

In [4]:
categorical_features = []
numerical_features = []

for column in data.columns:
    dtype = str(data[column].dtype)
    if dtype in ["float64", "int64"]:
        numerical_features.append(column)
    else:
        categorical_features.append(column)

In [5]:
for categorical_feature in categorical_features:
    data[categorical_feature].fillna('missing', inplace=True)

In [6]:
for categorical_feature in categorical_features:
    le = LabelEncoder()
    data[categorical_feature] = le.fit_transform(data[categorical_feature])

## Getting x_train, x_test, y_train, y_test

In [7]:
x_train, x_test, y_train, y_test = train_test_split(data.values, 
                                                    targets.values.ravel(), 
                                                    test_size=0.3, 
                                                    random_state=2021,
                                                    stratify=targets.values)

In [8]:
print(x_train.shape, x_test.shape)

(13410, 12) (5748, 12)


In [9]:
print(y_train.shape, y_test.shape)

(13410,) (5748,)


# MLFlow workspace preparation and Use

In [10]:
mlflow_uri_from_aws = str(input("Cual es la URL de instancia de EC2"))
#"http://3.226.165.98:5000/"

In [11]:
mlflow.set_tracking_uri(mlflow_uri_from_aws)

In [12]:
tracking_uri = mlflow.get_tracking_uri()

In [13]:
tracking_uri

'http://3.226.165.98:5000/'

### Adjusting resources metadata

In [ ]:
import mlflow.data
import pandas as pd
from mlflow.data.pandas_dataset import PandasDataset

#Train dataset URL
dataset_source_url="https://www.kaggle.com/datasets/arashnic/hr-analytics-job-change-of-data-scientists?resource=download&select=aug_train.csv"

#Using mlflow.data.pandas_dataset, to log_input dataset instead of artifact
dataset = mlflow.data.pandas_dataset.from_pandas(data, source=dataset_source_url)

## Part1: Run preparation for Random Forest Model

In [ ]:
experiment_name_formlflow = str(input("Cual es el nombre de tu experimento ?:"))
"01B3_MFlow_LR_train"


In [ ]:
print(experiment_name_formlflow)

In [ ]:
from mlflow import MlflowClient

# Create an experiment with a name that is unique and case sensitive.
client = MlflowClient()

#experiment_name_formlflow = "01Basic_MFlow_LR_train"
# Create an experiment name, which must be unique and case sensitive
experiment_id = client.create_experiment(
    experiment_name_formlflow
)

# Fetch experiment metadata information
experiment = client.get_experiment(experiment_id)
print("Name: {}".format(experiment.name))
print("Experiment_id: {}".format(experiment.experiment_id))
print("Artifact Location: {}".format(experiment.artifact_location))
print("Tags: {}".format(experiment.tags))
print("Lifecycle_stage: {}".format(experiment.lifecycle_stage))

In [ ]:
# start run here

## Checking runs

In [27]:
from mlflow.entities import ViewType

run = MlflowClient().search_runs(
    experiment_ids="23",
    filter_string="",
    run_view_type=ViewType.ACTIVE_ONLY,
    order_by=["metrics.accuracy DESC"],
)

print(len(run))
print(run[0])

1
<Run: data=<RunData: metrics={'accuracy': 0.7352122477383438,
 'auc': 0.7208662878564284,
 'f1': 0.5658870507701085,
 'precision': 0.4785335262904004,
 'recall': 0.6922540125610607}, params={'algo': 'Logistic Regression',
 'class_weight': 'balanced',
 'compute': 'local',
 'dataset': 'kaggle-dataset',
 'dataset_path': '../data/aug_train.csv"',
 'dataset_url': 'https://www.kaggle.com/datasets/arashnic/hr-analytics-job-change-of-data-scientists?resource=download&select=aug_train.csv',
 'dataset_version': '1.0',
 'max_iter': '1000'}, tags={'mlflow.log-model.history': '[{"run_id": "a44df8eaac0f4248a23b24b6bae2aa0f", '
                             '"artifact_path": "model", "utc_time_created": '
                             '"2023-07-13 16:13:25.602876", "flavors": '
                             '{"python_function": {"model_path": "model.pkl", '
                             '"predict_fn": "predict", "loader_module": '
                             '"mlflow.sklearn", "python_version": "3.8.3

In [53]:
#Getting dataset information
dataset_info = run.inputs.dataset_inputs[0].dataset
print(f"Dataset name: {dataset_info.name}")
print(f"Dataset digest: {dataset_info.digest}")
print(f"Dataset profile: {dataset_info.profile}")
print(f"Dataset schema: {dataset_info.schema}")

Dataset name: dataset
Dataset digest: b5ad9a5d
Dataset profile: {"num_rows": 19158, "num_elements": 229896}
Dataset schema: {"mlflow_colspec": [{"type": "string", "name": "city"}, {"type": "double", "name": "city_development_index"}, {"type": "string", "name": "gender"}, {"type": "string", "name": "relevent_experience"}, {"type": "string", "name": "enrolled_university"}, {"type": "string", "name": "education_level"}, {"type": "string", "name": "major_discipline"}, {"type": "string", "name": "experience"}, {"type": "string", "name": "company_size"}, {"type": "string", "name": "company_type"}, {"type": "string", "name": "last_new_job"}, {"type": "long", "name": "training_hours"}]}


In [54]:
#Checking last active run
run = mlflow.last_active_run()
run.info.run_id

'd5927bf6e30d472595068813b30ffc80'

## Part2: Loading model from artifact to use

# Conclusions

# References

# Other key functions and Info